In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
# df = pd.read_csv('./datasets/train.csv')
df = pd.read_csv('./datasets/test.csv')

In [3]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [4]:
df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [5]:
df_clean = df[['Lot Area',
               'Street',   # Secondary, removing from first model, value split of 2044 / 7
               'Land Contour',   # Value split of 1843 / 85 / 80 / 43 
               'Neighborhood',   
               'Condition 1',
               'Condition 2',   # Secondary, removing from first model, value split of 2025 (Normal) / 26 (other values)
               'Bldg Type', 
               'House Style',
               'Overall Cond',   # Secondary variable to investigate
               'Year Built', 
               'Year Remod/Add', 
               'Roof Style',   # Secondary variable to investigate
               'Exter Cond', 
               'Foundation',   # Secondary variable to investigate
               'Bsmt Cond',   # Secondary, removing from first model, 1834 (Typicals) / 92 (Good | Excellent) / 70 (Fair | Poor)
               'BsmtFin Type 1',   # Secondary variable to investigate
               'BsmtFin Type 2',   # Secondary variable to investigate 
               'Total Bsmt SF', 
               'Heating',   # Secondary, removing from first model, value split of 2018 (GasA) / 33 (other values)
               'Central Air',   # Secondary variable to investigate   
               'Gr Liv Area', 
               'Full Bath', 
               'Half Bath', 
               'Bedroom AbvGr', 
               'Kitchen AbvGr',   # Secondary variable to investigate
               'Kitchen Qual', 
               'Fireplaces',   # Secondary variable to investigate
               'Garage Type', 
               'Garage Cars',
               'Paved Drive',   # Secondary, removing from first model, 1861 (Paved) / 39 (Partial) / 151 (Dirt/Gravel)
               'Pool QC',   # Consider removing from first model, only 9 houses with pools
#                'SalePrice',
               'Id',
               'Yr Sold'
               ]].copy()



df_clean = df_clean.rename(columns = {'Lot Area':'lot_area',
                                      'Street':'street', 
                                      'Land Contour':'land_cont', 
                                      'Neighborhood':'neighborhood', 
                                      'Condition 1':'cond_1', 
                                      'Condition 2':'cond_2', 
                                      'Bldg Type':'bldg_type', 
                                      'House Style':'style',
                                      'Overall Cond':'overall_cond', 
                                      'Year Built':'yr_built', 
                                      'Year Remod/Add':'yr_remodeled',
                                      'Roof Style':'roof_style',
                                      'Exter Cond':'exter_cond', 
                                      'Foundation':'foundation',
                                      'Bsmt Cond':'bsmt_cond', 
                                      'BsmtFin Type 1':'bsmt_fin_1',
                                      'BsmtFin Type 2':'bsmt_fin_2', 
                                      'Total Bsmt SF':'bsmt_sf', 
                                      'Heating':'heat', 
                                      'Central Air':'cent_air',
                                      'Gr Liv Area':'gr_liv_area', 
                                      'Full Bath':'full_bath', 
                                      'Half Bath':'half_bath', 
                                      'Bedroom AbvGr':'bedrooms_gr',
                                      'Kitchen AbvGr':'kitchen',
                                      'Kitchen Qual':'kitch_qual',
                                      'Fireplaces':'fireplaces',
                                      'Garage Type':'garage_type',
                                      'Garage Cars':'garage_car_size',
                                      'Paved Drive':'paved_drive', 
                                      'Pool QC':'pool_qual',
#                                       'SalePrice':'sale_price',
                                      'Id':'Id',
                                      'Yr Sold':'year_sold'
                                      })

In [6]:
# df_clean['fe_ov_cond_gr_liv_area'] = df_clean['overall_cond'] * df_clean['gr_liv_area']

In [7]:
# df_clean['fe_yr_blt_gr_liv_area'] = df_clean['yr_built'] * df_clean['gr_liv_area']

In [8]:
df_clean['fe_bed_full_bath'] = df_clean['bedrooms_gr'] * df_clean['full_bath']

In [9]:
# Used .describe() to check mean/min/max values and ensure all numerical columns weren't missing data

df_clean.describe()

,lot_area,overall_cond,yr_built,yr_remodeled,bsmt_sf,gr_liv_area,full_bath,half_bath,bedrooms_gr,kitchen,fireplaces,garage_car_size,Id,year_sold,fe_bed_full_bath
count,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000,878.000000
mean,10307.033030,5.566059,1970.492027,1984.417995,1034.357631,1496.914579,1.539863,0.398633,2.879271,1.047836,0.618451,1.742597,1444.749431,2007.825740,4.618451
std,10002.674602,1.128903,30.395895,20.450725,413.446291,506.468967,0.559060,0.505927,0.830712,0.223966,0.669571,0.750391,850.838263,1.327861,2.548320
min,1477.000000,1.000000,1880.000000,1950.000000,0.000000,407.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2006.000000,0.000000
25%,7297.250000,5.000000,1954.000000,1967.000000,789.000000,1114.000000,1.000000,0.000000,2.000000,1.000000,0.000000,1.000000,692.250000,2007.000000,3.000000
50%,9446.000000,5.000000,1972.000000,1992.000000,975.000000,1436.000000,2.000000,0.000000,3.000000,1.000000,1.000000,2.000000,1433.000000,2008.000000,4.000000
75%,11589.000000,6.000000,1999.750000,2003.000000,1247.000000,1779.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000,2197.500000,2009.000000,6.000000
max,215245.000000,9.000000,2010.000000,2010.000000,2630.000000,4476.000000,4.000000,2.000000,6.000000,3.000000,3.000000,4.000000,2928.000000,2010.000000,24.000000


In [10]:
# Check how mayn null values in each column

df_clean.isnull().sum()

lot_area              0
street                0
land_cont             0
neighborhood          0
cond_1                0
cond_2                0
bldg_type             0
style                 0
overall_cond          0
yr_built              0
yr_remodeled          0
roof_style            0
exter_cond            0
foundation            0
bsmt_cond            25
bsmt_fin_1           25
bsmt_fin_2           25
bsmt_sf               0
heat                  0
cent_air              0
gr_liv_area           0
full_bath             0
half_bath             0
bedrooms_gr           0
kitchen               0
kitch_qual            0
fireplaces            0
garage_type          44
garage_car_size       0
paved_drive           0
pool_qual           874
Id                    0
year_sold             0
fe_bed_full_bath      0
dtype: int64

In [11]:
# Investigate bsmt_cond and bsmt_sf for null bsmt_sf entry

null_bsmt_cond = df_clean[df_clean['bsmt_cond'].isnull()]
null_bsmt_sf = df_clean[df_clean['bsmt_sf'].isnull()]
# null_bsmt_cond[['bsmt_cond', 'bsmt_sf']]
null_bsmt_sf[['bsmt_cond', 'bsmt_sf']]
# df_clean.iloc[1327]

,bsmt_cond,bsmt_sf


In [12]:
# Checking .value_counts() for columns

df_clean['bsmt_cond'].value_counts()

TA    781
Fa     39
Gd     33
Name: bsmt_cond, dtype: int64

In [13]:
df_clean['bsmt_cond'].sort_values()

439     Fa
27      Fa
287     Fa
270     Fa
583     Fa
      ... 
621    NaN
625    NaN
654    NaN
702    NaN
771    NaN
Name: bsmt_cond, Length: 878, dtype: object

In [14]:
# Checking datatypes for all columns

df_clean.dtypes

lot_area             int64
street              object
land_cont           object
neighborhood        object
cond_1              object
cond_2              object
bldg_type           object
style               object
overall_cond         int64
yr_built             int64
yr_remodeled         int64
roof_style          object
exter_cond          object
foundation          object
bsmt_cond           object
bsmt_fin_1          object
bsmt_fin_2          object
bsmt_sf              int64
heat                object
cent_air            object
gr_liv_area          int64
full_bath            int64
half_bath            int64
bedrooms_gr          int64
kitchen              int64
kitch_qual          object
fireplaces           int64
garage_type         object
garage_car_size      int64
paved_drive         object
pool_qual           object
Id                   int64
year_sold            int64
fe_bed_full_bath     int64
dtype: object

In [15]:
# plt.figure(figsize=(8,5))
# plt.scatter(df_clean['overall_cond'], df_clean['sale_price'], alpha=.25);  # Look at Pandas boxplot

In [16]:
# plt.figure(figsize=(8,5))
# plt.scatter(df_clean['yr_remodeled'], df_clean['sale_price'], alpha=.25);

In [17]:
# plt.figure(figsize=(8,5))
# plt.scatter(df_clean['gr_liv_area'], df_clean['sale_price'], alpha=.25);

In [18]:
# plt.figure(figsize=(8,5))
# plt.scatter(df_clean['bsmt_sf'], df_clean['sale_price'], alpha=.25);

In [19]:
# Created has_pool column to use binary data if house has pool and disregard condition/quality

df_clean['has_pool'] = np.where(df_clean['pool_qual'].isnull(), 0, 1)
df_clean['has_pool'].value_counts()

0    874
1      4
Name: has_pool, dtype: int64

In [20]:
# Replace null values
# https://www.geeksforgeeks.org/python-pandas-dataframe-fillna-to-replace-null-values-in-dataframe/

df_clean['bsmt_sf'] = df_clean["bsmt_sf"].fillna(0)
df_clean["bsmt_cond"] = df_clean["bsmt_cond"].fillna('None')
df_clean["bsmt_fin_1"] = df_clean["bsmt_fin_1"].fillna('None')
df_clean["bsmt_fin_2"] = df_clean["bsmt_fin_2"].fillna('None')
df_clean["garage_type"] = df_clean["garage_type"].fillna('None')

In [21]:
# Replace garage_car_size (NaN) with median value (mean and median are essentially the same)

df_clean['garage_car_size'].notnull().median()
df_clean["garage_car_size"] = df_clean["garage_car_size"].fillna(1) 

In [22]:
# Drop the pool_qual column, already converted to has_pool

df_clean.drop(columns=['pool_qual'], inplace=True)

In [23]:
# Check for remaining null values

df_clean.isnull().sum()

lot_area            0
street              0
land_cont           0
neighborhood        0
cond_1              0
cond_2              0
bldg_type           0
style               0
overall_cond        0
yr_built            0
yr_remodeled        0
roof_style          0
exter_cond          0
foundation          0
bsmt_cond           0
bsmt_fin_1          0
bsmt_fin_2          0
bsmt_sf             0
heat                0
cent_air            0
gr_liv_area         0
full_bath           0
half_bath           0
bedrooms_gr         0
kitchen             0
kitch_qual          0
fireplaces          0
garage_type         0
garage_car_size     0
paved_drive         0
Id                  0
year_sold           0
fe_bed_full_bath    0
has_pool            0
dtype: int64

In [24]:
# Enter cond_2 values of [RRNn, RRAn, RRNe, RRAe] for cond_1 values if cond_1 values [Artery, Feedr, Norm, PosN, or PosA]

for num in range(len(df_clean)):
    
    cond_tst_1a = df_clean['cond_2'][num] == 'RRAn'
    cond_tst_1b = df_clean['cond_2'][num] == 'RRAe'
    cond_tst_2a = df_clean['cond_2'][num] == 'RRNn'
    cond_tst_2b = df_clean['cond_2'][num] == 'RRNe'
    cond_tst_2c = df_clean['cond_1'][num] != 'RRAn'
    cond_tst_2d = df_clean['cond_1'][num] != 'RRAe'    
    gets_replaced = df_clean['cond_1'][num]
    does_replacing = df_clean['cond_2'][num]
     
    if cond_tst_1a or cond_tst_1b:
        df_clean.replace(gets_replaced, does_replacing, inplace=True)
    elif (cond_tst_2a or cond_tst_2b) and (cond_tst_2c or cond_tst_2d):
        df_clean.replace(gets_replaced, does_replacing, inplace=True)
        
        
df_clean.head()

,lot_area,street,land_cont,neighborhood,cond_1,cond_2,bldg_type,style,overall_cond,yr_built,...,kitchen,kitch_qual,fireplaces,garage_type,garage_car_size,paved_drive,Id,year_sold,fe_bed_full_bath,has_pool
0,9142,Pave,Lvl,OldTown,Norm,Norm,2fmCon,2Story,8,1910,...,2,Fa,0,Detchd,1,Y,2658,2006,8,0
1,9662,Pave,Lvl,Sawyer,Norm,Norm,Duplex,1Story,4,1977,...,2,TA,0,Attchd,2,Y,2718,2006,12,0
2,17104,Pave,Lvl,Gilbert,Norm,Norm,1Fam,2Story,5,2006,...,1,Gd,1,Attchd,2,Y,2414,2006,6,0
3,8520,Pave,Lvl,OldTown,Norm,Norm,1Fam,1Story,6,1923,...,1,TA,0,Detchd,2,N,1989,2007,2,0
4,9500,Pave,Lvl,NAmes,Norm,Norm,1Fam,1Story,5,1963,...,1,TA,2,Attchd,2,Y,625,2009,3,0


In [25]:
# Mapping small neighborhoods to larger adjacent neighborhoods

df_clean['neighborhood'] = df_clean['neighborhood'].map({'NAmes':'NAmes',
                                                         'CollgCr':'CollgCr',
                                                         'OldTown':'OldTown',
                                                         'Edwards':'Edwards',
                                                         'Somerst':'Somerst',
                                                         'NridgHt':'NridgHt',
                                                         'Gilbert':'Gilbert',
                                                         'Sawyer':'Sawyer',
                                                         'SawyerW':'SawyerW',
                                                         'Mitchel':'Mitchel',
                                                         'BrkSide':'BrkSide',
                                                         'Crawfor':'Crawfor',
                                                         'IDOTRR':'IDOTRR',
                                                         'Timber':'Timber',
                                                         'NoRidge':'NoRidge',
                                                         'StoneBr':'StoneBr',
                                                         'SWISU':'SWISU',
                                                         'ClearCr':'ClearCr',
                                                         'MeadowV':'MeadowV',
                                                         'Blmngtn':'Blmngtn',
                                                         'BrDale':'BrDale',
                                                         'Veenker':'Veenker',
                                                         'NPkVill':'NPkVill',
                                                         'Blueste':'Crawfor',
                                                         'Greens':'Somerst',
                                                         'GrnHill':'Timber',
                                                         'Landmrk':'Somerst'
                                                        })

In [26]:
# Mapping style to groups

df_clean['style'] = df_clean['style'].map({'1Story':'1Story',
                                           '2Story':'2Story',
                                           '1.5Fin':'Fin',
                                           'SLvl':'SLvl',
                                           'SFoyer':'SFoyer',
                                           '2.5Unf':'Unfin',
                                           '1.5Unf':'Unfin',
                                           '2.5Fin':'Fin'
                                          })

In [27]:
# Mapping overall_cond to combine three lowest values

df_clean['overall_cond'] = df_clean['overall_cond'].map({1:3,
                                                         2:3,
                                                         3:3,
                                                         4:4,
                                                         5:5,
                                                         6:6,
                                                         7:7,
                                                         8:8,
                                                         9:9
                                                        })

In [28]:
# Mapping roof_style so small groups go to Other

df_clean['roof_style'] = df_clean['roof_style'].map({'Gable':'Gable',
                                                     'Hip':'Hip',
                                                     'Flat':'Other',
                                                     'Gambrel':'Other',
                                                     'Mansard':'Other',
                                                     'Shed':'Other'
                                                    })

In [29]:
# Mapping exter_cond to combine like categories

df_clean['exter_cond'] = df_clean['exter_cond'].map({'TA':'TA',
                                                     'Gd':'Gd',
                                                     'Fa':'Fa',
                                                     'Ex':'Gd',
                                                     'Po':'Fa'
                                                    })

In [30]:
# Mapping foundation to combine smaller categories to Other

df_clean['foundation'] = df_clean['foundation'].map({'PConc':'PConc',
                                                     'CBlock':'CBlock',
                                                     'BrkTil':'BrkTil',
                                                     'Slab':'Other',
                                                     'Stone':'Other',
                                                     'Wood':'Other'
                                                    })

In [31]:
# Mapping exter_cond to combine like categories

df_clean['bsmt_cond'] = df_clean['bsmt_cond'].map({'TA':'TA',
                                                   'Gd':'Gd',
                                                   'Fa':'Fa',
                                                   'Ex':'Gd',
                                                   'Po':'Fa',
                                                   'None':'None'
                                                    })

In [32]:
# Mapping heating to group non-gas options

df_clean['heat'] = df_clean['heat'].map({'GasA':'GasA',
                                         'GasW':'GasW',
                                         'Wall':'NonGas',
                                         'Grav':'NonGas',
                                         'OthW':'NonGas'
                                        })

In [33]:
# Mapping kitchen to groups

df_clean['kitchen'] = df_clean['kitchen'].map({1:1,
                                               0:1,
                                               3:2,
                                               2:2
                                              })

In [34]:
# Mapping fireplaces to groups

df_clean['fireplaces'] = df_clean['fireplaces'].map({0:0,
                                                     1:1,
                                                     2:2,
                                                     3:2,
                                                     4:2
                                                    })

In [35]:
# Mapping garage_car_size to groups

df_clean['garage_car_size'] = df_clean['garage_car_size'].map({0:0,
                                                               1:1,
                                                               2:2,
                                                               3:3,
                                                               4:3,
                                                               5:3
                                                              })

In [36]:
# Mapping kitch_qual to combine like categories

df_clean['kitch_qual'] = df_clean['kitch_qual'].map({'TA':'TA',
                                                     'Gd':'Gd',
                                                     'Fa':'Fa',
                                                     'Ex':'Ex',
                                                     'Po':'Fa',
                                                      })

In [37]:
# Mapping cond_1 to combine like categories

df_clean['cond_1'] = df_clean['cond_1'].map({'Norm':'Norm',
                                             'Feedr':'Artery',
                                             'Artery':'Artery',
                                             'RRAn':'RRA',
                                             'PosN':'Fa',
                                             'PosA':'Fa',
                                             'RRAe':'RRA',
                                             'RRNn':'RRN',
                                             'RRNe':'RRN'
                                            })

In [38]:
# Check value_counts for all categorical data

# street               # DONE          DON'T USE, 2044 / 7 feature split
# land_cont            # DONE          Max: 1843 / Min: 43
# neighborhood         # DONE          Max: 310 / Min: 1, 2, 3, 6; NEED TO CHECK MAP AND GROUP SMALL NUMBER
# cond_1               # DONE          Combined RRNn/RRNe, RRAn/RRAe
# cond_2               # DONE          DO NOT USE
# bldg_type            # DONE          Max: 1700 / Min: 46
# style                # DONE          COMBINE 1.5Fin and 2.5Fin, 1.5Unf and 2.5Unf
# overall_cond         # DONE          Max: 1168 / Min: 4; COMBINE 1, 2, 3 values
# roof_style           # DONE          Max: 1619 / Min: 3; DON'T USE AT FIRST
# exter_cond           # DONE          COMBINE Gd and Ex, Po and Fa
# foundation           # DONE          Max: 926 / Min: 2, 5; CONSIDER COMBINING Wood and Stone
# bsmt_cond            # DONE          COMBINE Gd and Ex, Po and Fa
# bsmt_fin_1           # DONE          Max: 615 / Min: 102
# bsmt_fin_2           # DONE          DON'T USE; Max: 1749 / Min: 23 
# bsmt_sf              # DONE          USE AS NUMERICAL
# heat                 # DONE          COMBINE NON-GAS METHODS (Wall, Grav, OthW)
# cent_air             # DONE          Y: 1910 / N: 141
# kitchen              # DONE          COMBINE 0 and 1, 2 and 3
# kitch_qual           # DONE          USE AS IS
# fireplaces           # DONE          COMBINE 2 and 3 and 4
# garage_type          # DONE          USE AS IS
# garage_car_size      # DONE          COMBINE 3 and 4 and 5
# paved_drive          # DONE          DON'T USE AT FIRST, USE AS IS
# has_pool             # DONE          0: 2042 / 1: 9
# sale_price           # DONE          Y target

df_clean['cond_1'].value_counts()

Norm      755
Artery     77
RRA        21
Fa         19
RRN         6
Name: cond_1, dtype: int64

In [39]:
df_clean.to_csv('./datasets/test_cleaned_fe3.csv')
# df_clean.to_csv('./datasets/train_cleaned_fe3.csv')

In [40]:
df_clean['cond_1'].head(50)

0       Norm
1       Norm
2       Norm
3       Norm
4       Norm
5       Norm
6       Norm
7       Norm
8       Norm
9       Norm
10      Norm
11      Norm
12      Norm
13    Artery
14      Norm
15    Artery
16      Norm
17      Norm
18        Fa
19    Artery
20      Norm
21    Artery
22      Norm
23      Norm
24      Norm
25      Norm
26    Artery
27    Artery
28    Artery
29    Artery
30      Norm
31      Norm
32      Norm
33      Norm
34      Norm
35       RRA
36      Norm
37      Norm
38      Norm
39      Norm
40      Norm
41      Norm
42      Norm
43      Norm
44      Norm
45      Norm
46      Norm
47      Norm
48      Norm
49    Artery
Name: cond_1, dtype: object